In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import sqlite3

# Define a function to fetch stock data and save to SQLite
def fetch_and_store_data(ticker, company_name, db_path):
    """
    Fetch stock data for a specific ticker and save it to an SQLite database.

    Parameters:
    ticker (str): The stock ticker symbol (e.g., 'XOM').
    company_name (str): The full company name.
    db_path (str): Path to the SQLite database file.
    """
    try:
        # Fetch data from yfinance
        stock_data = yf.download(ticker, start='2015-01-01', end='2022-12-31', progress=True)
        
        # Flatten multi-level column headers (if any)
        stock_data.columns = [col[0] if isinstance(col, tuple) else col for col in stock_data.columns]
        
        # Add additional columns for the company and ticker
        stock_data['Company'] = company_name
        stock_data['Ticker'] = ticker
        
        # Reset the index to make 'Date' a column
        stock_data.reset_index(inplace=True)
        
        # Connect to SQLite database
        with sqlite3.connect(db_path) as conn:
            # Save data to the database (append mode)
            stock_data.to_sql('stocks', conn, if_exists='append', index=False)
        
        print(f"Data for {company_name} ({ticker}) saved to SQLite database.")
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")

In [2]:
# List of companies and their tickers
stocks = [
    {'ticker': 'XOM', 'company': 'Exxon Mobil'},
    {'ticker': 'CVX', 'company': 'Chevron'},
    {'ticker': 'COP', 'company': 'ConocoPhillips'},
    {'ticker': 'T', 'company': 'AT&T'},
    {'ticker': 'TMUS', 'company': 'T-Mobile'},
    {'ticker': 'VZ', 'company': 'Verizon'}
]

# Path to SQLite database file
db_path = 'stocks_data.db'

# Fetch and store data for each stock
for stock in stocks:
    fetch_and_store_data(stock['ticker'], stock['company'], db_path)

# Verify data in the SQLite database
print("\nVerifying stored data:")
with sqlite3.connect(db_path) as conn:
    query = "SELECT DISTINCT Ticker, Company FROM stocks"
    stored_data = pd.read_sql(query, conn)
    print(stored_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Data for Exxon Mobil (XOM) saved to SQLite database.
Data for Chevron (CVX) saved to SQLite database.



[*********************100%***********************]  1 of 1 completed


Data for ConocoPhillips (COP) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed


Data for AT&T (T) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed


Data for T-Mobile (TMUS) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed

Data for Verizon (VZ) saved to SQLite database.

Verifying stored data:
  Ticker         Company
0    XOM     Exxon Mobil
1    CVX         Chevron
2    COP  ConocoPhillips
3      T            AT&T
4   TMUS        T-Mobile
5     VZ         Verizon
